# Open Interface and Rotavapor
Using Open Interface on a Rotavapor to get and display measured sensor data

## Connect to the Rotavapor

In [ ]:
import requests
import urllib3
from os import path

address = "10.93.152.210"
user = "rw"
password = "vCT3SuJq"

auth = (user, password)
base_url = f"https://{address}/api/v1"
info_endpoint = base_url + "/info"
process_endpoint = base_url + "/process"
rootcert = "root_cert.crt"

# configure http client session
session = requests.Session()
session.auth = auth
if path.isfile(rootcert):
    # if there is a root certificate file we'll use that one
    session.verify = rootcert
else:
    # ... if not, we'll ignore certificate errors
    print("Root certificate missing. Disabling certificate checks...")
    session.verify = False
    # this would cause warnings on the console which we disable here
    urllib3.disable_warnings()

# verify that this is a Rotavapor
info_resp = session.get(info_endpoint)
if info_resp.status_code != 200:
    raise Exception("Unexpected status code when getting device info", info_resp.status_code)
info_msg = info_resp.json()
system_name = info_msg["systemName"]
print(f"Connected to {system_name}")
if info_msg["systemClass"] != "Rotavapor":
    raise Exception(f"This is not a Rotavapor: {address}")

## Start measuring and displaying the temperature of the bath

See https://matplotlib.org/2.1.1/api/_as_gen/matplotlib.pyplot.plot.html for colors and marker styles, etc.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
from datetime import datetime

fig, ax = plt.subplots()
ln, = plt.plot([], [], 'b.')
xdata, ydata = [], []

# min and max values for the x axis
time_min = np.datetime64(datetime.now(), 'm')
time_max = time_min + np.timedelta64(1, 'm')

def read_temperature():
    # read temperature
    proc_resp = session.get(process_endpoint)
    
    if proc_resp.status_code != 200:
        raise Exception("Unexpected status code when polling process data", proc_resp.status_code)
    
    proc_msg = proc_resp.json()
    
    return proc_msg["heating"]["act"]

def init():
    global time_min, time_max
    
    fig.suptitle("Measurement started: " + datetime.now().strftime('%d.%m.%Y %H:%M:%S'), fontsize=10)

    # xaxis: minutes are major ticks, seconds are minor ticks
    ax.xaxis.set_major_locator(mdates.MinuteLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%M'))
    ax.xaxis.set_minor_locator(mdates.SecondLocator())
    
    # xaxis: set limit to current minute to next minute 
    ax.set_xlim(time_min, time_max)
    
    # xaxis: format is minutes:seconds
    ax.format_xdata = mdates.DateFormatter('%M:%S')
    
    # yaxis: set limit to 0 .. 100°C
    ax.set_ylim(0, 100)

    return ln,

def update(frame):
    global time_min, time_max
    
    # update xlim everytime the minute changes on the pc clock
    if np.datetime64(datetime.now(), 'm') >= time_max:
        time_max = np.datetime64(datetime.now(), 'm') + np.timedelta64(1, 'm')
        ax.set_xlim(time_min, time_max)
    
    # update the data with the new clock time and the measured temperature from the roti
    xdata.append(datetime.now())
    ydata.append(read_temperature())
    ln.set_data(xdata, ydata)
    
    # display the current roti bath temperature in the title of the graph
    ax.set_title("{0}°C".format(ydata[-1]))
    
    # display the current date and time as the label of the x axis
    plt.xlabel(datetime.now().strftime('%d.%m.%Y %H:%M:%S'))
    
    return ln,
    
ani = FuncAnimation(fig, update, init_func=init, interval=1000)
plt.show()

In [ ]:
# Pause measurement
ani.event_source.stop()

In [ ]:
# Resume measurement
ani.event_source.start()